In [9]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('/home/jupyter-23521027/refresh-bert/data/sbert/indobert-large-p1_dense_trained-epoch14', device='cuda:0')

In [15]:
import os
import json
import pandas as pd

folder_path = '/home/jupyter-23521027/refresh-bert/data/liputan6_data/canonical/test' 

file_list = os.listdir(folder_path)
json_data_list = []
file_list = os.listdir(folder_path)

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name) 

    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            json_data_list.append(json_data)

df = pd.DataFrame(json_data_list)
df['sentences'] = df['clean_article'].apply(lambda x: [' '.join(sentence) for sentence in x])
df.tail(1)

,id,url,clean_article,clean_summary,extractive_summary,sentences
10971,26102,https://www.liputan6.com/news/read/26102/eks-t...,"[[Liputan6, ., com, ,, Jakarta, :, Bekas, taha...","[[Forum, Eks, Tahanan, Politik, meminta, Presi...","[0, 1]","[Liputan6 . com , Jakarta : Bekas tahanan poli..."


In [16]:
file_path = "/home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.test.indobertlargep1.sbert"

existing_ids = []
with open(file_path, 'r') as file:
    lines = file.readlines()

    for line in lines:
        line = line.strip()

        if line and ' ' not in line:
            existing_ids.append(line.split("-")[1])
            
print(existing_ids[:5])
print(len(existing_ids))

FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.test.indobertlargep1.sbert'

In [ ]:
print(existing_ids[-5:])


In [ ]:
from datetime import datetime

MAX_DOC_LENGTH = 100
MAX_SENT_EMBEDDING_LENGTH = 250
DATASET_SIZE = 999999

now = datetime.now()
file = open(file_path , 'a')
file.write("\n")

for i, row in df.iterrows():
    if str(row['id']) in existing_ids:
        continue

    file.write(f"liputan6-{str(row['id'])}\n")

    for sent_txt in row["sentences"][:MAX_DOC_LENGTH]:
        sent_embedding = model.encode(sent_txt, convert_to_numpy=True).tolist()[:MAX_SENT_EMBEDDING_LENGTH]

        for element in sent_embedding:
            file.write(str(element) + " ")
            
        file.write("\n")
    
    if (i + 1) % 1000 == 0:
        print(f"processing {i + 1} with time: {datetime.now() - now}")
    
    if (i + 1) == DATASET_SIZE:
        break
    
    file.write("\n")
        
file.close()

processing 1000 with time: 0:06:41.660696
processing 2000 with time: 0:13:19.785658
processing 3000 with time: 0:20:05.822751
processing 4000 with time: 0:26:37.018799
processing 5000 with time: 0:33:35.373205
processing 6000 with time: 0:40:23.815908
processing 7000 with time: 0:47:14.951817


In [14]:
!cat /home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.validation.indobertlargep1.sbert | tail -c 100000

45117688179016 -0.7577534317970276 0.08604022115468979 0.09003457427024841 -0.1260419487953186 -0.4284418523311615 -0.46087464690208435 -0.1674603372812271 0.1312067061662674 0.5182692408561707 0.11082927882671356 0.09080483764410019 -0.28867655992507935 0.00681701535359025 -0.45550620555877686 0.5420043468475342 -0.3843809962272644 -0.5563986897468567 0.1570235788822174 0.07031027227640152 0.04737665131688118 -0.4477348029613495 -0.41422387957572937 -0.1459565907716751 -0.42105501890182495 -0.371599406003952 0.21840940415859222 -0.46460485458374023 -0.15039090812206268 0.5891585350036621 0.1986069232225418 -0.23601973056793213 -0.6694307327270508 0.518442690372467 -0.008709531277418137 0.2208574265241623 0.08382602035999298 0.21806715428829193 -0.046894244849681854 -0.4559406042098999 0.07049617916345596 -0.1757090985774994 0.42301973700523376 0.11187237501144409 0.021792136132717133 -0.5147843956947327 0.4960694909095764 0.6221344470977783 0.39303478598594666 -0.12775199115276337 0.5